Introduction

In [ ]:
'''
goal: to analysis flight booking data form database to get the future booking status of next day, next 7, 15 and 30 day

process flow:
1. extract data from database
2. transform data
3. filter data
4. process data
5. send data as json to depict using charts

'''

Data Extraction

In [ ]:
import pandas as pd
import pymysql

# print("---------START: Reading Data---------")
mydb = pymysql.connect(
    host = "host_name",
    user = "user_name",
    database = "db_name",
    password = "password")

column_header = ['route','flight_no','flight_date', 'total_seat','total_booked','plf','generated_at']

mycursor = mydb.cursor()
mycursor.execute('select route,flight_no,flight_date, total_seat, total_booked,plf,generated_at from table_name')

df = pd.DataFrame(mycursor.fetchall())
df.columns = column_header

# print("---------END: Reading Data---------")

Data Transformation

In [ ]:
# print("---------START: Transforming Data---------")

# Convert the columns to desired object.
df['flight_date']   = pd.to_datetime(df['flight_date'])
df['generated_at']  = pd.to_datetime(df['generated_at'])
df['total_seat']    = df['total_seat'].astype(int)
df['total_booked']  = df['total_booked'].astype(int)
df['plf']           = df['plf'].astype(float).round(2)

# print("---------END: Transforming Data---------")

Data Filter

In [ ]:
# Get today's date to fiter out future booking data
today_date = pd.to_datetime('today')

# Filter the data to only include flights with a flight date from today's date.
future_flights  = df[df['flight_date'] >= today_date]
future_flights_grouped = future_flights.loc[future_flights.groupby([ 'flight_no', 'flight_date','route'])['generated_at'].idxmax()]

Data Process

In [ ]:
# print("---------START: Analyzing Data---------")
import datetime
import numpy as np

if future_flights_grouped.empty:

    # Return the summary statistics.
    summary_statistics = {
        'status'                          : 0,
        'total_bookings'                  : 0,
        'total_capacity'                  : 0,
        'average_booking_load_1'          : 0,
        'average_booking_load_7'          : 0,
        'average_booking_load_15'         : 0,
        'average_booking_load_30'         : 0,
        'date_0'                          : 0,
        'date_1'                          : 0,
        'date_7'                          : 0,
        'date_15'                         : 0,
        'date_30'                         : 0,
        'last_update_date'                : 0,
        'most_popular_specific_flight'    : 0,
        'least_popular_specific_flight'   : 0,
        'most_popular_overall_route_1'    : 0,
        'least_popular_overall_route_1'   : 0,
        'most_popular_overall_route_7'    : 0,
        'least_popular_overall_route_7'   : 0,
        'most_popular_overall_route_15'   : 0,
        'least_popular_overall_route_15'  : 0,
        'most_popular_overall_route_30'   : 0,
        'least_popular_overall_route_30'  : 0,
        'sorted_routes_1'                 : 0,
        'sorted_routes_7'                 : 0,
        'sorted_routes_15'                : 0,
        'sorted_routes_30'                : 0,
        'json_dict_1'                     : 0,
        'json_dict_7'                     : 0,
        'json_dict_15'                    : 0,
        'json_dict_30'                    : 0
        
    }
else:

    # Calculate the total number of bookings.
    total_bookings = future_flights_grouped['total_booked'].sum()
    total_capacity = future_flights_grouped['total_seat'].sum()
    
    # Filter the data for the next 1,7,15,30 days
    current_date = pd.to_datetime('today')
    last_update_date = max(future_flights_grouped['generated_at'])
    
    end_date_tomorrow = current_date + datetime.timedelta(days=1)
    next_1_days_data = future_flights_grouped[(future_flights_grouped['flight_date'] > current_date) & (future_flights_grouped['flight_date'] <= end_date_tomorrow)]

    end_date_7 = current_date + datetime.timedelta(days=7)
    next_7_days_data = future_flights_grouped[(future_flights_grouped['flight_date'] > current_date) & (future_flights_grouped['flight_date'] <= end_date_7)]

    end_date_15 = current_date + datetime.timedelta(days=15)
    next_15_days_data = future_flights_grouped[(future_flights_grouped['flight_date'] > current_date) & (future_flights_grouped['flight_date'] <= end_date_15)]

    end_date_30 = current_date + datetime.timedelta(days=30)
    next_30_days_data = future_flights_grouped[(future_flights_grouped['flight_date'] > current_date) & (future_flights_grouped['flight_date'] <= end_date_30)]

    
    plf_100_df_1  = next_1_days_data[next_1_days_data['plf'] >= 100]
    plf_100_df_7  = next_7_days_data[next_7_days_data['plf'] >= 100]
    plf_100_df_15 = next_15_days_data[next_15_days_data['plf'] >= 100]
    plf_100_df_30 = next_30_days_data[next_30_days_data['plf'] >= 100]

    # Group by 'route', 'flight_no', 'flight_date' and get unique combinations
    unique_combinations_1  = plf_100_df_1[['route', 'flight_no', 'flight_date']].drop_duplicates()
    unique_combinations_7  = plf_100_df_7[['route', 'flight_no', 'flight_date']].drop_duplicates()
    unique_combinations_15 = plf_100_df_15[['route', 'flight_no', 'flight_date']].drop_duplicates()
    unique_combinations_30 = plf_100_df_30[['route', 'flight_no', 'flight_date']].drop_duplicates()
    
    # Merge with another_df to get rows that match the grouped rows
    result_df_1  = future_flights.merge(unique_combinations_1, on=['route', 'flight_no', 'flight_date'], how='inner').sort_values(by='generated_at', ascending=True)
    result_df_7  = future_flights.merge(unique_combinations_7, on=['route', 'flight_no', 'flight_date'], how='inner').sort_values(by='generated_at', ascending=True)
    result_df_15 = future_flights.merge(unique_combinations_15, on=['route', 'flight_no', 'flight_date'], how='inner').sort_values(by='generated_at', ascending=True)
    result_df_30 = future_flights.merge(unique_combinations_30, on=['route', 'flight_no', 'flight_date'], how='inner').sort_values(by='generated_at', ascending=True)
    
    # Create a dictionary to store separate DataFrames
    dataframes_dict_1  = {}
    dataframes_dict_7  = {}
    dataframes_dict_15 = {}
    dataframes_dict_30 = {}
    
    # Iterate over unique combinations to create separate DataFrames
    for _, row in unique_combinations_1.iterrows():
        route, flight_no, flight_date = row['route'], row['flight_no'], row['flight_date']
        key = f"{route}_{flight_no}_{flight_date}"
        dataframes_dict_1[key] = result_df_1[(result_df_1['route'] == route) &
                                            (result_df_1['flight_no'] == flight_no) &
                                            (result_df_1['flight_date'] == flight_date)]
        
    # Convert each DataFrame in the dictionary to JSON and store in a new dictionary
    json_dict_1 = {key: df.to_json(orient='records') for key, df in dataframes_dict_1.items()}
    
    for _, row in unique_combinations_7.iterrows():
        route, flight_no, flight_date = row['route'], row['flight_no'], row['flight_date']
        key = f"{route}_{flight_no}_{flight_date}"
        dataframes_dict_7[key] = result_df_7[(result_df_7['route'] == route) &
                                            (result_df_7['flight_no'] == flight_no) &
                                            (result_df_7['flight_date'] == flight_date)]
        
    # Convert each DataFrame in the dictionary to JSON and store in a new dictionary
    json_dict_7 = {key: df.to_json(orient='records') for key, df in dataframes_dict_7.items()}
    
    for _, row in unique_combinations_15.iterrows():
        route, flight_no, flight_date = row['route'], row['flight_no'], row['flight_date']
        key = f"{route}_{flight_no}_{flight_date}"
        dataframes_dict_15[key] = result_df_15[(result_df_15['route'] == route) &
                                            (result_df_15['flight_no'] == flight_no) &
                                            (result_df_15['flight_date'] == flight_date)]
        
    # Convert each DataFrame in the dictionary to JSON and store in a new dictionary
    json_dict_15 = {key: df.to_json(orient='records') for key, df in dataframes_dict_15.items()}
    
    for _, row in unique_combinations_30.iterrows():
        route, flight_no, flight_date = row['route'], row['flight_no'], row['flight_date']
        key = f"{route}_{flight_no}_{flight_date}"
        dataframes_dict_30[key] = result_df_30[(result_df_30['route'] == route) &
                                            (result_df_30['flight_no'] == flight_no) &
                                            (result_df_30['flight_date'] == flight_date)]
        
    # Convert each DataFrame in the dictionary to JSON and store in a new dictionary
    json_dict_30 = {key: df.to_json(orient='records') for key, df in dataframes_dict_30.items()}
    
    # Calculate the average booking load across all flights.
    average_booking_load_1  = np.mean(next_1_days_data['plf']  )
    average_booking_load_7  = np.mean(next_7_days_data['plf']  )
    average_booking_load_15 = np.mean(next_15_days_data['plf'] )
    average_booking_load_30 = np.mean(next_30_days_data['plf'] )
    
    df_sorted = future_flights_grouped.sort_values(by='plf', ascending=False)

    # Get the best 3 and worst 3 specific flight numbers
    best_3_specific = df_sorted.head(3)[['flight_no','flight_date','plf']]
    worst_3_specific = df_sorted.tail(3)[['flight_no','flight_date','plf']]

    # Group by 'route' and calculate the average load and total number of flights
    route_summary_1 = next_1_days_data.groupby('route').agg(
        average_load = ('plf', 'mean'),
        total_flights = ('flight_date', 'count'),
        total_flight_no = ('flight_no', 'count')
    ).reset_index()
    
    # Sort the flights based on average load
    sorted_routes_1 = route_summary_1.sort_values(by='average_load', ascending=False)

    # Get the best 3 and worst 3 flight numbers
    best_3_overall_route_1 = sorted_routes_1.head(3)
    worst_3_overall_route_1 = sorted_routes_1.tail(3)[::-1]
    
    # Group by 'route' and calculate the average load and total number of flights
    route_summary_7 = next_7_days_data.groupby('route').agg(
        average_load = ('plf', 'mean'),
        total_flights = ('flight_date', 'count'),
        total_flight_no = ('flight_no', 'count')
    ).reset_index()
    
    # Sort the flights based on average load
    sorted_routes_7 = route_summary_7.sort_values(by='average_load', ascending=False)

    # Get the best 3 and worst 3 flight numbers
    best_3_overall_route_7 = sorted_routes_7.head(3)
    worst_3_overall_route_7 = sorted_routes_7.tail(3)[::-1]
    
    # Group by 'route' and calculate the average load and total number of flights
    route_summary_15 = next_15_days_data.groupby('route').agg(
        average_load = ('plf', 'mean'),
        total_flights = ('flight_date', 'count'),
        total_flight_no = ('flight_no', 'count')
    ).reset_index()
    
    # Sort the flights based on average load
    sorted_routes_15 = route_summary_15.sort_values(by='average_load', ascending=False)

    # Get the best 3 and worst 3 flight numbers
    best_3_overall_route_15 = sorted_routes_15.head(3)
    worst_3_overall_route_15 = sorted_routes_15.tail(3)[::-1]
    
    # Group by 'route' and calculate the average load and total number of flights
    route_summary_30 = next_30_days_data.groupby('route').agg(
        average_load = ('plf', 'mean'),
        total_flights = ('flight_date', 'count'),
        total_flight_no = ('flight_no', 'count')
    ).reset_index()
    
    # Sort the flights based on average load
    sorted_routes_30 = route_summary_30.sort_values(by='average_load', ascending=False)

    # Get the best 3 and worst 3 flight numbers
    best_3_overall_route_30 = sorted_routes_30.head(3)
    worst_3_overall_route_30 = sorted_routes_30.tail(3)[::-1]

    # Return the summary statistics.
    summary_statistics = {
        'status'                          : 1,
        'total_bookings'                  : total_bookings.astype(str),
        'total_capacity'                  : total_capacity.astype(str),
        'average_booking_load_1'          : average_booking_load_1.astype(str),
        'average_booking_load_7'          : average_booking_load_7.astype(str),
        'average_booking_load_15'         : average_booking_load_15.astype(str),
        'average_booking_load_30'         : average_booking_load_30.astype(str),
        'date_0'                          : current_date.strftime('%d-%m-%Y'),
        'date_1'                          : end_date_tomorrow.strftime('%d-%m-%Y'),
        'date_7'                          : end_date_7.strftime('%d-%m-%Y'),
        'date_15'                         : end_date_15.strftime('%d-%m-%Y'),
        'date_30'                         : end_date_30.strftime('%d-%m-%Y'),
        'last_update_date'                : last_update_date.strftime('%d-%m-%Y'),
        'most_popular_specific_flight'    : best_3_specific.to_json(orient ='records'),
        'least_popular_specific_flight'   : worst_3_specific.to_json(orient ='records'),
        'most_popular_overall_route_1'    : best_3_overall_route_1.to_json(orient ='records'),
        'least_popular_overall_route_1'   : worst_3_overall_route_1.to_json(orient ='records'),
        'most_popular_overall_route_7'    : best_3_overall_route_7.to_json(orient ='records'),
        'least_popular_overall_route_7'   : worst_3_overall_route_7.to_json(orient ='records'),
        'most_popular_overall_route_15'   : best_3_overall_route_15.to_json(orient ='records'),
        'least_popular_overall_route_15'  : worst_3_overall_route_15.to_json(orient ='records'),
        'most_popular_overall_route_30'   : best_3_overall_route_30.to_json(orient ='records'),
        'least_popular_overall_route_30'  : worst_3_overall_route_30.to_json(orient ='records'),
        'sorted_routes_1'                 : sorted_routes_1.to_json(orient ='records'),
        'sorted_routes_7'                 : sorted_routes_7.to_json(orient ='records'),
        'sorted_routes_15'                : sorted_routes_15.to_json(orient ='records'),
        'sorted_routes_30'                : sorted_routes_30.to_json(orient ='records'),
        'json_dict_1'                     : json_dict_1,
        'json_dict_7'                     : json_dict_7,
        'json_dict_15'                    : json_dict_15,
        'json_dict_30'                    : json_dict_30
        
    }
    
# print("---------END: Analyzing Data---------")

Data Dump

In [ ]:
import json

json_data       = json.dumps(summary_statistics)